In [54]:
    from abc import ABC, abstractmethod
    import numpy as np

    """ 
    Base class for sharing 
    attributes and functions of FD 
    """
    class FiniteDifferences(object):

        def __init__(
            self, S0, K, r=0.05, T=1, 
            sigma=0, Smax=1, M=1, N=1, is_put=False
        ):
            self.S0 = S0
            self.K = K
            self.r = r
            self.T = T
            self.sigma = sigma
            self.Smax = Smax
            self.M, self.N = M, N
            self.is_call = not is_put

            self.i_values = np.arange(self.M)
            self.j_values = np.arange(self.N)
            self.grid = np.zeros(shape=(self.M+1, self.N+1))
            self.boundary_conds = np.linspace(0, Smax, self.M+1)

        @abstractmethod
        def setup_boundary_conditions(self):
            raise NotImplementedError('Implementation required!')

        @abstractmethod
        def setup_coefficients(self):
            raise NotImplementedError('Implementation required!')

        @abstractmethod
        def traverse_grid(self):
            """  Iterate the grid backwards in time"""
            raise NotImplementedError('Implementation required!')

        @abstractmethod
        def interpolate(self):
            """ Use piecewise linear interpolation on the initial
            grid column to get the closest price at S0.
            """
            return np.interp(
                self.S0, self.boundary_conds, self.grid[:,0])
        @property
        def dS(self):
            return self.Smax/float(self.M)

        @property
        def dt(self):
            return self.T/float(self.N)
        def price(self):
            self.setup_boundary_conditions()
            self.setup_coefficients()
            self.traverse_grid()
            return self.interpolate()

In [55]:
import numpy as np

""" 
Explicit method of Finite Differences 
"""
class FDExplicitEu(FiniteDifferences):
    def setup_boundary_conditions(self):
        if self.is_call:
            self.grid[:,-1] = np.maximum(
                0, self.boundary_conds - self.K)
#                 t=T时刻所有的价格 max（S-K,0）
            self.grid[-1,:-1] = (self.Smax-self.K) * \
                np.exp(-self.r*self.dt*(self.N-self.j_values))
            self.grid[0,:-1]=0
#     t<T时Smax的V价格。最底下那一行。
        else:
            self.grid[:,-1] = np.maximum(
                0, self.K-self.boundary_conds)
            self.grid[0,:-1] = self.K * \
                np.exp(-self.r*self.dt*(self.N-self.j_values))
            self.grid[-1,:-1] =0 
# 默认Smax比行权价格大
    def setup_coefficients(self):
        self.a = 0.5*self.dt*((self.sigma**2) *
                              (self.i_values**2) -
                              self.r*self.i_values)
        self.b = 1 - self.dt*((self.sigma**2) *
                              (self.i_values**2) +
                              self.r)
        self.c = 0.5*self.dt*((self.sigma**2) *
                              (self.i_values**2) +
                              self.r*self.i_values)
    def traverse_grid(self):
        for j in reversed(self.j_values):
            for i in range(self.M)[2:]:
                self.grid[i,j] = \
                    self.a[i]*self.grid[i-1,j+1] +\
                    self.b[i]*self.grid[i,j+1] + \
                    self.c[i]*self.grid[i+1,j+1]

# European

In [56]:

import numpy as np
import scipy.linalg as linalg

""" 
Crank-Nicolson method of Finite Differences 
"""
class FDCnEu(FDExplicitEu):
    def setup_coefficients(self):
        self.alpha = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) - \
            self.r*self.i_values)
        self.beta = -self.dt*0.5*(
            (self.sigma**2)*(self.i_values**2) + self.r)
        self.gamma = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) +
            self.r*self.i_values)
        self.M1 = -np.diag(self.alpha[2:self.M], -1) + \
                  np.diag(1-self.beta[1:self.M]) - \
                  np.diag(self.gamma[1:self.M-1], 1)
        self.M2 = np.diag(self.alpha[2:self.M], -1) + \
                  np.diag(1+self.beta[1:self.M]) + \
                  np.diag(self.gamma[1:self.M-1], 1)
    def traverse_grid(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.M1)

        for j in reversed(range(self.N)):
            x1 = linalg.solve(
                L, np.dot(self.M2, self.grid[1:self.M, j+1]))
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2

In [57]:
option = FDCnEu(50, 50, r=0.1, T=5./12.,
        sigma=0.4, Smax=100, M=100, N=1000, is_put=True)
print(option.price())

4.072238354486828


In [58]:
option = FDCnEu(50, 50, r=0.1, T=5./12., 
        sigma=0.4, Smax=100, M=80, N=100, is_put=True)
print(option.price())

4.070145703042844


# Barrier

Down and Out

In [59]:

import numpy as np

"""
Price a down-and-out option by the Crank-Nicolson
method of finite differences.
"""
class FDCnDo(FDCnEu):

    def __init__(
        self, S0, K, r=0.05, T=1, sigma=0, 
        Sbarrier=0, Smax=1, M=1, N=1, is_put=False
    ):
        super(FDCnDo, self).__init__(
            S0, K, r=r, T=T, sigma=sigma,
            Smax=Smax, M=M, N=N, is_put=is_put
        )
        self.barrier = Sbarrier
        self.boundary_conds = np.linspace(Sbarrier, Smax, M+1)
        self.i_values = self.boundary_conds/self.dS
        
    def setup_boundary_conditions(self):
        if self.is_call:
            self.grid[:,-1] = np.maximum(
                0, self.boundary_conds - self.K)
#                 t=T时刻所有的价格 max（S-K,0）
            self.grid[-1,:-1] = (self.Smax-self.K) * \
                np.exp(-self.r*self.dt*(self.N-self.j_values))
            self.grid[0,:-1]=np.maximum((self.barrier-self.K) * np.exp(-self.r*self.dt*(self.N-self.j_values)),0)
#     t<T时Smax的V价格。最底下那一行。
        else:
            self.grid[:,-1] = np.maximum(
                0, self.K-self.boundary_conds)
            self.grid[0,:-1] = self.K * \
                np.exp(-self.r*self.dt*(self.N-self.j_values))
            self.grid[-1,:-1] =0 
    @property
    def dS(self):
        return (self.Smax-self.barrier)/float(self.M)

In [60]:
    option = FDCnDo(50, 40, r=0.04, T=1, 
        sigma=0.3, Sbarrier=20, Smax=225, M=100, N=200)
    print(option.price())

12.943798519671073


In [61]:
    option = FDCnDo(150, 150, r=0.05, T=0.5, sigma=0.25, 
        Sbarrier=125, Smax=225, M=120, N=500)
    print(option.price())

10.355821895846653


In [62]:
option = FDCnDo(50, 50, r=0.1, T=5./12., 
        sigma=0.4, Sbarrier=40, Smax=100, M=120, N=500)
print(option.price())

5.49156055293479


In [63]:

import numpy as np

"""
Price a down-and-out option by the Crank-Nicolson
method of finite differences.
"""
class FDCnUo(FDCnEu):

    def __init__(
        self, S0, K, r=0.05, T=1, sigma=0, 
        Sbarrier=0, Smax=1, M=1, N=1, is_put=False
    ):
        super(FDCnUo, self).__init__(
            S0, K, r=r, T=T, sigma=sigma,
            Smax=Smax, M=M, N=N, is_put=is_put
        )
        self.barrier = Sbarrier
        self.boundary_conds = np.linspace(0, Sbarrier,  M+1)
        self.i_values = self.boundary_conds/self.dS
        
    def setup_boundary_conditions(self):
        if self.is_call:
            self.grid[:,-1] = np.maximum(
                0, self.boundary_conds - self.K)
#                 t=T时刻所有的价格 max（S-K,0）
            self.grid[-1,:-1] = (self.barrier-self.K) * \
                np.exp(-self.r*self.dt*(self.N-self.j_values))
            self.grid[0,:-1]=0
        else:
            self.grid[:,-1] = np.maximum(
                0, self.K-self.boundary_conds)
            self.grid[0,:-1] = self.K * \
                np.exp(-self.r*self.dt*(self.N-self.j_values))
            self.grid[-1,:-1] =0 
    @property
    def dS(self):
        return (self.barrier)/float(self.M)

In [64]:
option = FDCnUo(50, 60, r=0.02, T=0.5, 
        sigma=0.45, Sbarrier=80, Smax=70, M=100, N=100)
print(option.price())

0.8313243779199595
